In [126]:
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
import os
import requests
import PyPDF2
import io
from IPython.display import Markdown, display, update_display
import json

load_dotenv('../.env', override=True)

openai = OpenAI()

# 1. Paper Summarizer in Portuguese

In [102]:
def message_gpt(url):
    content = get_pdf_content(url)
    messages = [
        {'role': 'system',
         'content': "You will summarize the user's paper in Brazilian Portuguese. The language should be easy and informal."},
        
        {'role': 'user',
         'content': f'Paper content:\n {content} \n\n Summarization:'}
    ]

    stream = openai.chat.completions.create(
        messages = messages,
        model='gpt-4o-mini',
        stream=True
    )

    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result


In [63]:
view = gr.Interface(fn=message_gpt, 
                    inputs=[gr.Textbox(label="Paper URL")],
                    outputs=gr.Textbox(label="Br Translation"),
                    flagging_mode="never")

view.launch(share=True)

* Running on local URL:  http://127.0.0.1:7882
* Running on public URL: https://bd65fd69d075b25519.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


# 2. Chat Interface

In [103]:
def chat(message, history):
    
    system_message = "You will always try to contradict the user."
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model='gpt-4o-mini', messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [104]:
gr.ChatInterface(chat, type='messages').launch()

* Running on local URL:  http://127.0.0.1:7895

To create a public link, set `share=True` in `launch()`.


# Tools

In [105]:
data = {'shoe': '$100', 
        'bag': '$32',
        'shirt': '$50'}

def get_price(item):
    item = item.lower()
    if item in data:
        return data[item]
    return "I don't know"

In [121]:
price_function = {
    "name": "get_price",
    "description": "Gets item's price.",
    "parameters": {
        "type": "object",
        "properties": {
            "item": {
                "type": "string",
                "description": "an object for sale in a clothing store, such as a hat or a shoe. it should be a single word."
            }
        },
    "required": ["item"],
    "additionalProperties": False
    }
}

tools = [{'type': 'function', 'function': price_function}]

In [99]:
def chat(message, history):
    
    system_message = "You are a useful assistant. Reply briefly. If you don't know something, tell the user you don't know it."
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    response = openai.chat.completions.create(model='gpt-4o-mini', messages=messages, tools=tools)

    if response.choices[0].finish_reason == 'tool_calls':
        message = response.choices[0].message
        tool_call = message.tool_calls[0]
        arguments = json.loads(tool_call.function.arguments)
        item = arguments['item']
        response = {
            'role': 'tool',
            'content': json.dumps({'item': item, 'price': get_price(item)}),
            'tool_call_id': message.tool_calls[0].id
        }
        messages.append(message)
        messages.append(response)
        print(response)
        response = openai.chat.completions.create(model='gpt-4o-mini', messages=messages)
        
    return response.choices[0].message.content
    

In [100]:
gr.ChatInterface(chat, type='messages').launch()

* Running on local URL:  http://127.0.0.1:7894

To create a public link, set `share=True` in `launch()`.


{'role': 'tool', 'content': '{"item": "shoe", "price": "$100"}', 'tool_call_id': 'call_2oZWaaTPHwLOu0EQYelZU47V'}
{'role': 'tool', 'content': '{"item": "handbag", "price": "I don\'t know"}', 'tool_call_id': 'call_o1OurG2m7IUqvHBnIG4xFCdr'}


# Multimodal Agents

In [135]:
import base64

from PIL import Image

client = OpenAI()

def get_image(food):

    # Request the image as base64
    response = client.images.generate(
        model="dall-e-3",
        prompt=food,
        size="1024x1024",
        response_format="b64_json"
    )

    image_base64 = response.data[0].b64_json
    image_bytes = base64.b64decode(image_base64)
    image = Image.open(io.BytesIO(image_bytes))
    return image

In [127]:
def create_image(food):
    return f'<{food}>'

In [128]:
chef = {
    "name": "chef",
    "description": "Gets a food description",
    "parameters": {
        "type": "object",
        "properties": {
            "food": {
                "type": "string",
                "description": "the description of a food or beverage."
            }
        },
    "required": ["item"],
    "additionalProperties": False
    }
}

tools = [{'type': 'function', 'function': chef}]

In [138]:
def chat(message, history):
    
    system_message = """
        You are the assistant to a virtual restaurant. 
        When the user asks for a food or beverage, no matter how crazy, you will provide him.
        """
    
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]

    response = openai.chat.completions.create(model='gpt-4o-mini', messages=messages, tools=tools)

    if response.choices[0].finish_reason == 'tool_calls':
        message = response.choices[0].message
        tool_call = message.tool_calls[0]
        arguments = json.loads(tool_call.function.arguments)
        food = arguments['food']
        response = {
            'role': 'assistant',
            'content': get_image(food),
            'tool_call_id': message.tool_calls[0].id
        }
        messages.append(response)
        response = openai.chat.completions.create(model='gpt-4o-mini', messages=messages)
        
    return response.choices[0].message.content
    

In [139]:
gr.ChatInterface(chat, type='messages').launch()

* Running on local URL:  http://127.0.0.1:7901

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Applications/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Applications/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Applications/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 2088, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Applications/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 1633, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Applications/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/utils.py", line 850, in async_wrapper
    

In [140]:
import gradio as gr

# Define the chatbot function
def respond(message, history):
    if "cat" in message.lower():
        # Reply with text + image in the same message
        return [("Here’s a cat I found!", "cat.png")]
    elif "dog" in message.lower():
        # Reply with only an image
        return [(None, "dog.png")]
    else:
        # Normal text-only reply
        return [("You said: " + message, None)]

# Create the ChatInterface
demo = gr.ChatInterface(
    fn=respond,
    chatbot=gr.Chatbot(height=500),
    textbox=gr.Textbox(placeholder="Ask me something..."),
    title="Chatbot with Images"
)

demo.launch()

/Applications/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/components/chatbot.py:282: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(
/Applications/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/chat_interface.py:317: UserWarning: The gr.ChatInterface was not provided with a type, so the type of the gr.Chatbot, 'tuples', will be used.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7902

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Applications/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Applications/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Applications/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 2088, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Applications/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 1633, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Applications/anaconda3/envs/llms/lib/python3.11/site-packages/gradio/utils.py", line 850, in async_wrapper
    